[Mediapipe mesh website](https://google.github.io/mediapipe/solutions/face_mesh#python-solution-api)

[Face landmark model ](https://arxiv.org/abs/1907.06724)

In [12]:
import cv2
import mediapipe as mp
import os

In [13]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

output_path = os.path.abspath(os.getcwd())

In [18]:
# For static images:
IMAGE_FILES = ['./images/paralysis_test.jpg']
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=2, refine_landmarks=True, min_detection_confidence=0.5) as face_mesh:
    for idx, file in enumerate(IMAGE_FILES):
        image = cv2.imread(file)
        
        # Convert the BGR image to RGB before processing.
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Print and draw face mesh landmarks on the image.
        if not results.multi_face_landmarks:
            continue
            
        annotated_image = image.copy()
        
        for face_landmarks in results.multi_face_landmarks:
            # print('face_landmarks:', face_landmarks)
            mp_drawing.draw_landmarks(
              image=annotated_image,
              landmark_list=face_landmarks,
              connections=mp_face_mesh.FACEMESH_TESSELATION,
              landmark_drawing_spec=None,
              connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(
              image=annotated_image,
              landmark_list=face_landmarks,
              connections=mp_face_mesh.FACEMESH_CONTOURS,
              landmark_drawing_spec=None,
              connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(
              image=annotated_image,
              landmark_list=face_landmarks,
              connections=mp_face_mesh.FACEMESH_IRISES,
              landmark_drawing_spec=None,
              connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())
  
        cv2.imwrite(output_path + '/out/annotated_image' + str(idx) + '.png', annotated_image)

/home/robbedec/repos/ugent/thesis-inwe/notebooks


In [1]:
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        # Draw the face mesh annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()

NameError: name 'mp_drawing' is not defined

In [1]:
from dlib import get_frontal_face_detector
from dlib import shape_predictor
from dlib import rectangle

import cv2
import os
import numpy as np
import sys

In [9]:
detector = get_frontal_face_detector()
predictor = shape_predictor('../mee_shape_predictor_68_face_landmarks.dat')

image = cv2.imread('../../images/paralysis_test.jpg')

In [11]:
height, width, d = image.shape                        
if d > 1:
    #transform to gray 
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#resize to speed up face dectection
#height, width = gray.shape[:2]  
newWidth=200
ScalingFactor=width/newWidth
newHeight=int(height/ScalingFactor)
smallImage=cv2.resize(gray, (newWidth, newHeight), interpolation=cv2.INTER_AREA)

In [12]:
rects = detector(smallImage,1)

In [16]:
results = []

if len(rects) == 0 : 
    #if no face detected then try again with the full size image
    rects = detector(gray,1)            
        
if len(rects) != 0:   
    #now we have only one face in the image
    #function to obtain facial landmarks using dlib 
    #given an image and a face
    #rectangle
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy array
        
        print('face {}'.format(i))
        
        inter_shape = np.zeros((68,2),dtype=int)

        #adjust face position using the scaling factor
        mod_rect=rectangle(
                left=int(rect.left() * ScalingFactor), 
                top=int(rect.top() * ScalingFactor), 
                right=int(rect.right() * ScalingFactor), 
                bottom=int(rect.bottom() * ScalingFactor)
        )

        #predict facial landmarks 
        shape_dlib = predictor(image, mod_rect)   
        #shape_dlib = predictor(gray, rect) 
        
        #transform shape object to np.matrix type
        for k in range(0,68):
            
            inter_shape[k] = (shape_dlib.part(k).x, shape_dlib.part(k).y)
            
            if inter_shape[k,0]<= 0: 
                inter_shape[k,0] = 1
                
            if inter_shape[k,1]<= 0: 
                inter_shape[k,1] = 1

        #position of the face in the image
        _boundingbox=[int(rect.left() * ScalingFactor), 
                           int(rect.top() * ScalingFactor),
                           int(rect.right() * ScalingFactor) - int(rect.left() * ScalingFactor),
                           int(rect.bottom() * ScalingFactor) - int(rect.top() * ScalingFactor)]
        
        results.append(np.copy(inter_shape))

    #it finished processing the face, now emit the results
    #return results

else: 
    # No detected faces
    print("shit")

face 0
face 1


In [19]:
print('List length: {}'.format(len(results)))
print(results[0].shape)

List length: 2
(68, 2)


In [28]:
for i, val in enumerate(results):
    for j, (x, y) in enumerate(val):
        print(j, x, y)
        cv2.circle(image, (x,y), 2, (255,0,0), cv2.FILLED)

0 353 161
1 351 187
2 353 213
3 356 238
4 363 261
5 374 284
6 392 301
7 412 315
8 437 322
9 461 317
10 483 303
11 502 286
12 516 265
13 524 243
14 527 219
15 529 193
16 529 167
17 370 146
18 381 135
19 395 130
20 411 132
21 426 138
22 465 132
23 479 126
24 495 125
25 509 131
26 517 145
27 443 156
28 443 173
29 442 189
30 441 207
31 421 217
32 431 220
33 441 223
34 451 220
35 460 218
36 387 156
37 396 149
38 409 150
39 420 161
40 407 161
41 395 161
42 464 162
43 477 149
44 490 149
45 499 158
46 491 162
47 478 163
48 394 246
49 410 239
50 428 238
51 437 240
52 448 238
53 466 241
54 480 246
55 469 258
56 454 265
57 439 266
58 424 265
59 407 259
60 400 246
61 423 246
62 438 247
63 452 245
64 475 248
65 454 257
66 438 259
67 423 258
0 49 165
1 49 192
2 53 218
3 59 242
4 67 266
5 79 287
6 99 302
7 122 312
8 148 315
9 172 306
10 192 291
11 209 273
12 220 253
13 226 230
14 227 205
15 228 180
16 227 156
17 61 145
18 70 133
19 85 128
20 103 129
21 120 134
22 158 122
23 173 114
24 189 111
25 205 

In [ ]:
cv2.imshow("Image", image)
cv2.waitKey(0)